In [1]:
import pandas as pd
import numpy as np
import spotipy
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from spotipy.oauth2 import SpotifyClientCredentials
from concurrent.futures import ThreadPoolExecutor, as_completed
warnings.filterwarnings('ignore')

SPOTIFY API

In [2]:
cid = 'd9219a18c2ed48e685ea287cbfcdda95'
secret = 'b9492ce5ac57427c9ee60be103779a29'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

RAW DATA

In [3]:
def classify_language(text):
    char_categories = {
        'ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅏㅑㅓㅕㅗㅛㅜㅠㅡㅣ': 0,
        'あいうえおかきくけこ': 1,
        'ñ': 2,
        'éèàùìò': 3,
        'əğ': 4,
        'абвгдежзийклмнопрстуфхцчшщъыьэюя': 5
    }

    text_lower = text.lower()
    for chars, category in char_categories.items():
        if any(char in text_lower for char in chars):
            return category
    return 6

In [4]:
df = pd.read_csv('final_data.csv')
df

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,language
0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.139,133.406,240166,3,6
1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.515,140.182,216387,4,6
2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.145,139.832,158960,4,6
3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.508,204.961,304293,4,6
4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.217,171.864,244320,4,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1160193,Kanye West,I Love It (& Lil Pump),4S8d14HvHb70ImctNgVzQQ,72,2018,chicago rap,0.901,0.522,2,-8.304,1,0.3300,0.0114,0.000000,0.2590,0.329,104.053,127946,4,6
1160194,Rihanna,FourFiveSeconds,78TTtXnFQPzwqlbtbwqN0y,80,2015,pop,0.582,0.272,2,-5.662,1,0.0501,0.8750,0.000000,0.1300,0.354,205.846,188238,4,6
1160195,Kanye West,FML,30Zcjs7pgEkmJA1lEbGSWT,64,2016,rap,0.527,0.415,5,-7.184,1,0.1310,0.3290,0.000000,0.1950,0.120,125.357,236120,4,6
1160196,¥$,PAPERWORK,2yyO7EKRr7c3txi4xCXUFk,77,2024,chicago rap,0.741,0.488,8,-5.308,1,0.0556,0.0703,0.000009,0.0821,0.186,120.839,145785,4,6


ENCODING & SCALING

In [5]:
label_encoder = LabelEncoder()
# df = pd.get_dummies(df, columns=['genre'], prefix=['genre'], dtype=int) - IF ONE HOT ENCODING
df['genre_encoded'] = label_encoder.fit_transform(df['genre'])

features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
            'instrumentalness', 'liveness', 'valence', 'tempo', 
            'time_signature', 'popularity', 'genre_encoded', 'language']

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])
# features += [col for col in df.columns if 'genre' in col] - IF ONE HOT ENCODING

CREATING THE FEATURE MATRIX

In [6]:
df['song_vector'] = df[features].values.tolist()
song_matrix = df[features].values
song_norms = np.linalg.norm(song_matrix, axis=1)
song_matrix.shape

(1160198, 13)

SONG INPUT

In [7]:
# def unknown_song_matrix(candidate_unknown_ids):
#     unknown_song_matrix = []
    
#     for i in candidate_unknown_ids:
#         try : 
#             results = sp.audio_features(i)
#             audio_data = [
#                 results[0]['danceability'],
#                 results[0]['energy'],
#                 results[0]["loudness"],
#                 results[0]['speechiness'],
#                 results[0]['acousticness'],
#                 results[0]['instrumentalness'],
#                 results[0]['liveness'],
#                 results[0]['valence'],
#                 results[0]['tempo'],
#                 results[0]['time_signature'],
#                 sp.track(i)['popularity']
#             ]
#         except Exception as e: 
#             print('EXCEPTION: Could not retrieve audio features for this song', e)
#             return 
#         genres = sp.artist(sp.track(i)['artists'][0]['id'])['genres']
#         if len(genres) == 0:
#             genre_encoded = 72
#         else:
#             genre_candidates = []
#             for genre in genres:
#                 try:
#                     genre_encoded = label_encoder.transform([genre])[0]
#                     genre_candidates.append(genre_encoded)
#                 except ValueError:
#                     pass

#             if genre_candidates:
#                 genre_encoded = np.mean(genre_candidates)
#             else:
#                 genre_encoded = 72
                
#         language = classify_language(sp.track(i)['name'])
#         audio_data.append(genre_encoded)
#         audio_data.append(language)
#         audio_data = np.array(audio_data).reshape(1, -1)
#         audio_data = scaler.transform(audio_data)
#         unknown_song_matrix.append(audio_data[0])

#     unknown_song_matrix = np.array(unknown_song_matrix)
#     return unknown_song_matrix

In [8]:
def fetch_track_data(track_id):
    try : 
        results = sp.audio_features(track_id)[0]
        audio_data = [
            results['danceability'],
            results['energy'],
            results["loudness"],
            results['speechiness'],
            results['acousticness'],
            results['instrumentalness'],
            results['liveness'],
            results['valence'],
            results['tempo'],
            results['time_signature'],
            sp.track(track_id)['popularity']
        ]
    except Exception as e: 
        print('EXCEPTION: Could not retrieve audio features for this song:', e)
        return 
    genres = sp.artist(sp.track(track_id)['artists'][0]['id'])['genres']
    if len(genres) == 0:
        genre_encoded = 72
    else:
        genre_candidates = []
        for genre in genres:
            try:
                genre_encoded = label_encoder.transform([genre])[0]
                genre_candidates.append(genre_encoded)
            except ValueError:
                pass

        if genre_candidates:
            genre_encoded = np.mean(genre_candidates)
        else:
            genre_encoded = 72
            
    language = classify_language(sp.track(track_id)['name'])
    audio_data.append(genre_encoded)
    audio_data.append(language)
    audio_data = np.array(audio_data).reshape(1, -1)
    audio_data = scaler.transform(audio_data)
    return audio_data[0]  

def unknown_song_matrix(candidate_unknown_ids):
    unknown_song_matrix = []
    with ThreadPoolExecutor(max_workers=50) as executor:
        future_to_track = {executor.submit(fetch_track_data, i): i for i in candidate_unknown_ids}
        for future in as_completed(future_to_track):
            data = future.result()
            if data is not None:
                unknown_song_matrix.append(data)

    return np.array(unknown_song_matrix)      

PLAYLIST INPUT & DISPLAY

In [12]:
playlist_link = "https://open.spotify.com/playlist/0NEtwypSBOv6KKMlgZOTp0?si=c0ec4727738f4bb9"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
playlist_data = sp.playlist(playlist_URI)
#retreive the profile picture of the playlist owner
user_id = playlist_data["owner"]["id"]
user_profile = sp.user(user_id)
user_profile_picture = user_profile["images"][0]["url"]


username = playlist_data["owner"]["display_name"]
playlist_name = playlist_data["name"]
number_of_tracks = playlist_data["tracks"]["total"]
playlist_cover_image = playlist_data["images"][0]["url"]
duration = sum([x["track"]["duration_ms"] for x in sp.playlist_tracks(playlist_URI)["items"]])
hours, minutes = duration//3600000, duration % 3600000 // 60000
duration = f"{hours} hours {minutes} minutes" if hours else f"{minutes} minutes" 
 

print(f"Username: {username}")
print(f"User Profile Picture: {user_profile_picture}")
print(f"Playlist Name: {playlist_name}")
print(f"Number of Tracks: {number_of_tracks}")
print(f"Playlist Cover Image: {playlist_cover_image}")
print(f"Duration: {duration}")

track_uris = [track["track"]["uri"] for track in playlist_data["tracks"]["items"]]
ids_playlist = [uri.split(":")[-1] for uri in track_uris]

for i in ids_playlist:
    ids_playlist.remove(i) if len(i) != 22 else None
    
print(user_id)

Username: Murad
User Profile Picture: https://i.scdn.co/image/ab67757000003b82a43719576a1a56ce2e931bbb
Playlist Name: random
Number of Tracks: 48
Playlist Cover Image: https://mosaic.scdn.co/640/ab67616d00001e0203e5cd20130bc76fad43f7b3ab67616d00001e020a6854a0a9c41d99b79ba5d5ab67616d00001e021917a0f3f4152622a040913fab67616d00001e024ae1c4c5c45aabe565499163
Duration: 2 hours 42 minutes
31jvkjkltwvspyklhzbujfjabqle


PLAYLIST RECOMMENDATION

In [10]:
candidate_known_ids = []
candidate_unknown_ids = []

for id in ids_playlist:
    try:
        input_song_vector_d = df[df['track_id'] == id]['song_vector'].values[0]
        candidate_known_ids.append(input_song_vector_d)
    except IndexError:
        candidate_unknown_ids.append(id)

candidate_known_ids = np.array(candidate_known_ids)
song_vectors = np.concatenate((candidate_known_ids, unknown_song_matrix(candidate_unknown_ids)), axis=0)
song_vectors = np.array([x for x in song_vectors if x is not None])
input_song_vector = np.mean(song_vectors, axis=0)

In [11]:
TOP = 20
similarities = np.dot(song_matrix, input_song_vector) / (song_norms * np.linalg.norm(input_song_vector))
sorted_indices = np.argsort(similarities)[::-1][:TOP]
top_songs = df.loc[sorted_indices, ['track_name', 'artist_name', 'track_id']] 
top_songs['similarity'] = similarities[sorted_indices]
top_songs = top_songs[~top_songs['track_id'].isin(ids_playlist)]

mp3_urls = []
spotify_urls = []
image_urls = []
ids = top_songs['track_id'].tolist()
results = sp.tracks(ids)
print(len(results['tracks']))

for track in results['tracks']:
    mp3_urls.append(track['preview_url'])
    spotify_urls.append(track['external_urls']['spotify'])
    image_urls.append(track['album']['images'][0]['url'])

top_songs['mp3_url'] = mp3_urls
top_songs['spotify_url'] = spotify_urls
top_songs['image_url'] = image_urls


top_songs

20


,track_name,artist_name,track_id,similarity,mp3_url,spotify_url,image_url
453648,Ecstasy - Mixed,Fabich,2PspwQLfDzLUOyaxQ7de5L,0.998367,https://p.scdn.co/mp3-preview/ccf1528127f0e074...,https://open.spotify.com/track/2PspwQLfDzLUOya...,https://i.scdn.co/image/ab67616d0000b273faa94a...
401180,RED,The Rose,1IguWzAOIxnmTZ7mqmKUwM,0.998207,https://p.scdn.co/mp3-preview/4594404eb7ef9b84...,https://open.spotify.com/track/1IguWzAOIxnmTZ7...,https://i.scdn.co/image/ab67616d0000b273e26b55...
1144181,Not Alone,Linkin Park,737yse38y2hsWQeNRrLaMY,0.998145,https://p.scdn.co/mp3-preview/3c6cc9b540dbb6ca...,https://open.spotify.com/track/737yse38y2hsWQe...,https://i.scdn.co/image/ab67616d0000b2734d6da9...
560950,CHANT (feat. Tones And I),Macklemore,5BgeAX4HuZdcuCqmu4pIMM,0.998067,https://p.scdn.co/mp3-preview/7bcc0da7ed9abb95...,https://open.spotify.com/track/5BgeAX4HuZdcuCq...,https://i.scdn.co/image/ab67616d0000b273497c7c...
562016,stacy's brother,Mad Tsai,7aTXnk6qGF8d4pZ51zDcaQ,0.997953,https://p.scdn.co/mp3-preview/c282092e08cae588...,https://open.spotify.com/track/7aTXnk6qGF8d4pZ...,https://i.scdn.co/image/ab67616d0000b27311e9a2...
293584,Drugs & Money - New Mix,Chase Atlantic,4SRyrhBJ5mq8fEa3bsdZn6,0.997838,https://p.scdn.co/mp3-preview/ac07afb53f5fc0eb...,https://open.spotify.com/track/4SRyrhBJ5mq8fEa...,https://i.scdn.co/image/ab67616d0000b2735a0c28...
564646,"Kanja Poovu Kannala (From ""Viruman"")",Yuvan Shankar Raja,78tblPcz2otRZd1VOeU7zz,0.997739,https://p.scdn.co/mp3-preview/21ceaae55f6a050d...,https://open.spotify.com/track/78tblPcz2otRZd1...,https://i.scdn.co/image/ab67616d0000b273ab415b...
515646,Ex For A Reason (with JT from City Girls),Summer Walker,1De7cOMrdUnVpjWBaowMg4,0.997722,None,https://open.spotify.com/track/1De7cOMrdUnVpjW...,https://i.scdn.co/image/ab67616d0000b273cd4fbe...
1003521,Replay,SHINee,1rp986nzkyAX1wFpxzbwlC,0.997706,https://p.scdn.co/mp3-preview/b6ab4149f2a5908f...,https://open.spotify.com/track/1rp986nzkyAX1wF...,https://i.scdn.co/image/ab67616d0000b273ff8a78...
457023,시작,Gaho,4kIpBfvK44bxqX7zo8K1oP,0.997663,None,https://open.spotify.com/track/4kIpBfvK44bxqX7...,https://i.scdn.co/image/ab67616d0000b273b86252...


SECOND APPROACH

In [115]:
playlist_link = "https://open.spotify.com/playlist/1Rbw9tvNFHHUCdtPuEnGEz?si=7d754acc8dc34b63"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
playlist_data = sp.playlist(playlist_URI)

username = playlist_data["owner"]["display_name"]
playlist_name = playlist_data["name"]
number_of_tracks = playlist_data["tracks"]["total"]
playlist_cover_image = playlist_data["images"][0]["url"]
duration = sum([x["track"]["duration_ms"] for x in sp.playlist_tracks(playlist_URI)["items"]])
hours, minutes = duration//3600000, duration % 3600000 // 60000
duration = f"{hours} hours {minutes} minutes" if hours else f"{minutes} minutes" 
 

print(f"Username: {username}")
print(f"Playlist Name: {playlist_name}")
print(f"Number of Tracks: {number_of_tracks}")
print(f"Playlist Cover Image: {playlist_cover_image}")
print(f"Duration: {duration}")

track_uris = [track["track"]["uri"] for track in playlist_data["tracks"]["items"]]
ids_playlist = [uri.split(":")[-1] for uri in track_uris]

for i in ids_playlist:
    ids_playlist.remove(i) if len(i) != 22 else None

Username: Murad
Playlist Name: indie
Number of Tracks: 13
Playlist Cover Image: https://mosaic.scdn.co/640/ab67616d00001e0296bc7d5bd0b6488de76598abab67616d00001e02a788765bdda04106bd7b0d29ab67616d00001e02b69321baf3e7031511fa0a93ab67616d00001e02efbe96040319db682b438f11
Duration: 41 minutes


In [116]:
candidate_known_ids = []
candidate_unknown_ids = []

for id in ids_playlist:
    try:
        input_song_vector_d = df[df['track_id'] == id]['song_vector'].values[0]
        candidate_known_ids.append(input_song_vector_d)
    except IndexError:
        candidate_unknown_ids.append(id)

candidate_known_ids = np.array(candidate_known_ids)
song_vectors = np.concatenate((candidate_known_ids, unknown_song_matrix(candidate_unknown_ids)), axis=0)
song_vectors = np.array([x for x in song_vectors if x is not None])
song_vectors_transpose = song_vectors.T
similarities_matrix = np.dot(song_matrix, song_vectors_transpose) / (song_norms[:, np.newaxis] * np.linalg.norm(song_vectors_transpose, axis=0))
similarities_matrix_transpose = similarities_matrix.T

In [117]:
# df_similarities = pd.DataFrame(similarities_matrix, index=df['track_id'], columns=ids_playlist)
# df_similarities

In [118]:
c = {}

for vector in similarities_matrix_transpose:
    indices = np.argsort(vector)[::-1][:5]
    values = vector[indices]
    for j in range(len(indices)):
        c[indices[j]] = values[j]
        
sorted_c = dict(sorted(c.items(), key=lambda item: item[1], reverse=True))

In [119]:
top_songs_new = pd.DataFrame()
indices = []
similarities = []
for key, value in sorted_c.items():
    if value < 1:
        indices.append(key)
        similarities.append(value)
    if len(indices) == 20:
        break

top_songs_new = df.loc[indices, ['track_name', 'artist_name', 'track_id']] 
top_songs_new['similarity'] = similarities
top_songs_new = top_songs_new[~top_songs_new['track_id'].isin(ids_playlist)]

mp3_urls = []
spotify_urls = []
image_urls = []
ids = top_songs_new['track_id'].tolist()
results = sp.tracks(ids)

for track in results['tracks']:
    mp3_urls.append(track['preview_url'])
    spotify_urls.append(track['external_urls']['spotify'])
    image_urls.append(track['album']['images'][0]['url'])

top_songs_new['mp3_url'] = mp3_urls
top_songs_new['spotify_url'] = spotify_urls
top_songs_new['image_url'] = image_urls

top_songs_new

,track_name,artist_name,track_id,similarity,mp3_url,spotify_url,image_url
387878,Feu rouge,Pépite,314LZAYaJXobwhsUapt0nJ,0.998973,https://p.scdn.co/mp3-preview/771ece1fb9fe5ec4...,https://open.spotify.com/track/314LZAYaJXobwhs...,https://i.scdn.co/image/ab67616d0000b273a26f63...
515738,The Moment I Knew (Taylor's Version),Taylor Swift,0NRHj8hDwwmSPaA41o379r,0.998307,None,https://open.spotify.com/track/0NRHj8hDwwmSPaA...,https://i.scdn.co/image/ab67616d0000b273318443...
354051,Birds,Imagine Dragons,6Tvzf3VEi16JMhAgOwdt2y,0.997746,None,https://open.spotify.com/track/6Tvzf3VEi16JMhA...,https://i.scdn.co/image/ab67616d0000b273da6f73...
1057254,Last Night on Earth,Green Day,5TpPSTItCwtZ8Sltr3vdzm,0.997682,https://p.scdn.co/mp3-preview/966749f64abdac65...,https://open.spotify.com/track/5TpPSTItCwtZ8Sl...,https://i.scdn.co/image/ab67616d0000b273c2ced3...
515599,Reckless,Madison Beer,5ajjAnNRh8bxFvaVHzpPjh,0.997680,https://p.scdn.co/mp3-preview/393eb37e58b0760f...,https://open.spotify.com/track/5ajjAnNRh8bxFva...,https://i.scdn.co/image/ab67616d0000b273fe32f3...
515764,What Other People Say,Sam Fischer,2h9TDNEXRhcDIV3fsoEVq9,0.997439,https://p.scdn.co/mp3-preview/12c47066042462eb...,https://open.spotify.com/track/2h9TDNEXRhcDIV3...,https://i.scdn.co/image/ab67616d0000b273027623...
569206,Satellite,Harry Styles,0rzaRSujxA0bKyjJl6vHYq,0.997437,https://p.scdn.co/mp3-preview/cff61b71737a5d63...,https://open.spotify.com/track/0rzaRSujxA0bKyj...,https://i.scdn.co/image/ab67616d0000b2732e8ed7...
569189,Glimpse of Us,Joji,6xGruZOHLs39ZbVccQTuPZ,0.996947,https://p.scdn.co/mp3-preview/12e4a7ef3f105142...,https://open.spotify.com/track/6xGruZOHLs39ZbV...,https://i.scdn.co/image/ab67616d0000b27308596c...
147040,Ennavale Adi Ennavale,Unnikrishnan,5KfLr5UF3iQN0ifFMQH0ol,0.996743,None,https://open.spotify.com/track/5KfLr5UF3iQN0if...,https://i.scdn.co/image/ab67616d0000b2733adcf2...
77593,Do I Wanna Know?,Arctic Monkeys,5FVd6KXrgO9B3JPmC8OPst,0.996536,https://p.scdn.co/mp3-preview/006bc465fe3d1c04...,https://open.spotify.com/track/5FVd6KXrgO9B3JP...,https://i.scdn.co/image/ab67616d0000b2734ae1c4...
